In [1]:
import pandas as pd

df = pd.read_csv('resumedata.csv')
df = df.astype(str)
df.head()

,accomplishments_segment,degree,education_segment,emails,job_titles,links,misc_segment,name,objectives_segment,phone,...,text,university_0,university_1,university_2,university_3,university_4,university_5,url,work_experience,work_segment
0,nan,"MS , B.TECH , PhD",nan,sutarvinayak2063@gmail.com,"full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...",nan,Prasad Nagar,nan,+919309161108,...,VINAYAK \nSUTAR \nLinkedIn \n+91 9309161108 ...,"national, mainpuri, uttar pradesh","college of arts, sundargarh, odisha","school of engineering, ernakulam, kerala",nan,nan,nan,https://training-resumes.s3.ap-south-1.amazona...,nan,nan
1,nan,"B.E , MS , M.TECH",M.Tech. Data Science\nJSS Science And Technolo...,pooj.vs@gmail.com,software engineer,pooj.vs,06/2017 – 07/2019\n07/2018 – 07/2019\nmailto:p...,Software Engineer,nan,7411786822,...,POOJA V S\nSoftware Engineer\nRich exposure to...,"government, kandhamal, odisha","vidya vardhaka college of engineering, mysore,...",nan,nan,nan,nan,https://training-resumes.s3.ap-south-1.amazona...,nan,Merchandising System' using PL/SQL.\nmaintain ...
2,Successfully cleared and completed the course ...,"MS , B.TECH , BS",Degree\nInstitute\nBoard/University\nYear\nPer...,khandelwal.ashwin5@gmail.com,nan,https://www.linkedin.com/in/ashwin-khandelwal-...,nan,Ashwin Khandelwal,"I seek challenging environment to work in, tha...",9462710290,...,Ashwin Khandelwal \n Email id – khand...,"narsee monjee institute of management studies,...",nan,nan,nan,nan,nan,https://training-resumes.s3.ap-south-1.amazona...,nan,Organization / Institute\nDescription\nBharat ...
3,nan,"B.E , MS , BS",nan,nan,"work from home , support analyst , software an...",nan,nan,Computer Engineering,nan,919930485336,...,Assessment\nQ1. Why should you be hired for th...,"master, medak, telangana",nan,nan,nan,nan,nan,https://training-resumes.s3.ap-south-1.amazona...,1.0,nan
4,nan,"MS , B.TECH , BS",nan,nan,"work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,nan, Python,nan,918010295832,...,Assessment\nQ1. Why should you be hired for th...,"greater noida institute of technology, gautam ...",nan,nan,nan,nan,nan,https://training-resumes.s3.ap-south-1.amazona...,nan,nan


In [2]:
# Combining the text columns
df['combined_text'] = df.apply(lambda x: ' '.join(x[['accomplishments_segment', 'degree', 'education_segment', 'job_titles', 'misc_segment', 'objectives_segment', 'projects_segment', 'skills', 'skills_segment', 
                                                     'work_experience', 'work_segment']]), axis=1)
df['combined_text'].head()

0    nan MS , B.TECH , PhD nan full stack developer...
1    nan B.E , MS , M.TECH M.Tech. Data Science\nJS...
2    Successfully cleared and completed the course ...
3    nan B.E , MS , BS nan work from home , support...
4    nan MS , B.TECH , BS nan work from home , clas...
Name: combined_text, dtype: object

In [3]:
#feature extraction 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['combined_text'])
X = pd.DataFrame(X.toarray(), columns=tfidf.get_feature_names_out())
X.head()

,00,000,01,02,03,04,05,06,07,08,...,zeplin,zero,zfs,zone,zones,zoning,zookeeper,zoom,zuul,zycus
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.337321,0.097336,0.038497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.024465,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#model

from sklearn.metrics.pairwise import cosine_similarity

def rank_resumes(job_description, resumes, tfidf_vectorizer):
    job_desc_tfidf = tfidf_vectorizer.transform([job_description])
    similarity_scores = cosine_similarity(job_desc_tfidf, resumes)
    ranked_indices = similarity_scores.argsort()[0][::-1]
    return ranked_indices

In [5]:
def display_ranked_resumes(ranked_indices, df):
    print("\nRanked Resumes:\n")
    for i, idx in enumerate(ranked_indices):
        resume = df.iloc[idx]
        print(f"Rank {i+1}: {resume['name']} - {resume['emails']}")
        print(f"Accomplishments: {resume['accomplishments_segment']}")
        print(f"Degree: {resume['degree']}")
        print(f"Job Titles: {resume['job_titles']}")
        print(f"Skills: {resume['skills']}\n")

In [7]:
#model evaluation

import numpy as np

def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])
labeled_data = [
    {"job_description": "Python developer with experience in machine learning", "relevant_resume": 10},
    {"job_description": "Data scientist with experience in R and Python", "relevant_resume": 5},
]

# Calculate MRR
reciprocal_ranks = []
for data in labeled_data:
    job_description = data["job_description"]
    relevant_resume = data["relevant_resume"]
    
    ranked_indices = rank_resumes(job_description, X, tfidf)
    rank_list = [1 if idx == relevant_resume else 0 for idx in ranked_indices]
    
    reciprocal_ranks.append(rank_list)

mrr = mean_reciprocal_rank(reciprocal_ranks)
print(f"Mean Reciprocal Rank (MRR): {mrr}")


Mean Reciprocal Rank (MRR): 0.0034497661175513523


In [8]:
# Hardcoded results


job_description = "Looking for a software engineer with experience in Python, machine learning, and web development. Must have a degree in computer science and strong problem-solving skills."
ranked_indices = rank_resumes(job_description, X, tfidf)
display_ranked_resumes(ranked_indices, df)
print("\nPredicted Ranked Resumes (Hardcoded):\n")
for i, idx in enumerate(ranked_indices[:5]):
    resume = df.iloc[idx]
    print(f"Rank {i+1}: {resume['name']} - {resume['emails']}")
    print(f"Accomplishments: {resume['accomplishments_segment']}")
    print(f"Degree: {resume['degree']}")
    print(f"Job Titles: {resume['job_titles']}")
    print(f"Skills: {resume['skills']}\n")


Ranked Resumes:

Rank 1: Senior Software - nan
Accomplishments: and indexing.

Degree: BS , B.E , MS , B.TECH , Ph.D
Job Titles: associate , devops , software developer , senior software engineer , full time , system engineer
Skills: automation , management skills , devops , interfaces , jenkins , telugu , css , root cause , soap , designation , developers , office , tamil , affirmative action , suggestions , mysql , web services , hindi , databases , cloud , docker , maintenance , technical manuals , windows , master data , json , english , authorization , troubleshooting , code review , resolutions , bootstrap , status , architecture , c# , postgresql , email , project , agile , time management , application , non-conforming , continuous integration , ci , xml , indexing , range , github , electronics , rest , estimates , servers , debugging , problem solving , building , languages , cerner , needs analysis , kannada , iterative , voices , google analytics , one , javascript , engin